In [69]:
import pandas as pd
import numpy as np

In [70]:
xls = pd.ExcelFile("Assignment_3_Data.xlsx")
df1 = pd.read_excel(xls, 'Firm Characteristics',skiprows=6,header=1) # Corporate/Financial Characteristics
df2 = pd.read_excel(xls, 'Aggregates',skiprows=7,header=1) # Aggregate summary data
df3 = pd.read_excel(xls, 'Stock Prices',header=1) # This is the time series data


In [71]:
df3.dropna(axis="columns", inplace=True)
df3 = df3.apply(pd.to_numeric, errors='coerce').dropna(axis="columns")

In [72]:
sample = df3.sample(n=50, axis=1)
sample

,CNO,PH,ANTM,VIVO,FBIO,FLXS,PRO,PCBK,EGHT,MRK,...,ASH,AEGN,AROC,CYH,DCO,CMD,KEY,IOSP,EMCI,VRTX
0,5.90701,70.48912,61.05436,15.14185,6.1000,12.38757,14.82,7.42710,3.39,31.54210,...,26.21889,16.15,5.15077,14.13369,12.92,12.47679,6.87689,25.11491,11.50862,32.23
1,5.87915,70.19403,61.38848,15.01481,6.2240,12.31585,14.82,7.30666,3.36,31.57505,...,26.53592,16.18,5.20338,14.07623,12.61,12.14856,6.91229,25.40808,11.40324,32.20
2,5.74912,70.58748,61.86709,14.92747,6.2744,12.44070,14.74,7.36287,3.29,31.90447,...,26.52687,16.17,4.84035,14.32247,12.88,12.07417,7.08045,25.99442,11.59181,33.12
3,5.71197,69.42503,63.87184,14.87983,6.0000,12.51154,14.92,7.14608,3.23,31.68211,...,26.87108,15.86,4.76143,15.02833,13.01,12.36738,7.06275,25.64794,11.37551,34.05
4,5.66553,70.48017,64.27820,14.89571,5.7500,12.74176,14.63,7.06578,3.14,31.61622,...,26.94354,15.94,4.71934,14.83134,12.90,12.81376,7.07160,25.66571,11.36996,35.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,24.42000,182.87000,195.64000,15.20000,3.9400,49.50000,25.05,26.90000,12.55,63.40000,...,66.85000,22.46,11.65000,6.44000,34.07,97.05000,18.45000,63.00000,29.07000,151.06
1462,24.70000,184.46000,195.25000,14.50000,3.8800,48.78000,25.28,27.05000,12.20,63.11000,...,67.65000,22.49,11.65000,6.33000,34.98,96.38000,18.46000,62.85000,28.98000,151.11
1463,24.82000,183.68000,205.71000,14.90000,3.6200,48.61000,25.12,26.95000,12.50,62.45000,...,67.68000,22.40,11.70000,6.17000,35.02,95.68000,18.32000,62.15000,29.22000,146.80
1464,24.48000,184.83000,206.91000,14.70000,3.6200,49.42000,24.73,27.80000,12.50,61.99000,...,68.40000,23.18,11.70000,5.82000,34.14,95.83000,18.52000,61.70000,29.55000,143.76


In [119]:
returns = df3.reset_index(drop=True).drop(["Date"], axis=1).astype(float).pct_change()
returns = returns.iloc[1:, :] # all but first row, because we can't get the returns of the first day
mean_returns = returns.mean() # Average DAILY returns 
cov_matrix = returns.cov() # Average DAILY convariance 

In [120]:
# Annualize the data
mu = (mean_returns + 1) ** 252 - 1
sigma = 252 * cov_matrix

n = mu.size # This is a 1D array

In [122]:
# Generate random portfolios
def mu_portfolio(w, mu):
    print(w)
    print(mu)
    return(w.T @ mu)

def sigma_portfolio(w, sigma):
    return(w.T @ sigma @ w)

def generate_random_portfolio(mu, sigma) -> dict:
    # Get random indexes
    num_stocks = 50
    idx = np.random.choice(np.array(range(len(mu))), num_stocks, replace=False)
    mu_sample = mu[idx]
    sigma_sample = sigma.iloc[idx, :]

    w = np.empty(0)
    for i in range(len(mu_sample)):
        w = np.append(w, np.random.uniform(0, 1))
    w = w / np.sum(w)
    w = np.random.permutation(w)
    mu0 = mu_portfolio(w, mu_sample)
    sigma0 = sigma_portfolio(w, sigma_sample)
    return({"w": w, "mu": mu0, "sigma": sigma0})

In [123]:
# Now we can generate a bunch of random portfolios
N = 10 * n
random_portfolios = [generate_random_portfolio(mu, sigma) for _ in range(N)]

TypeError: '(array([1103, 1006, 2022, 1393, 1610, 1883, 1119, 1055,  114, 1706,  255,
        745,  765, 1185,  809, 2146, 1969,  881, 1936, 1689, 1428, 2181,
       1258,  891,  361, 1913,   94, 2069, 1640, 1159, 1001, 1222, 1869,
        386,  479, 1676, 1709,  804, 1872, 1384,  239, 1108,  688,  562,
       1324,  566,  682,  183, 1331, 1935]), slice(None, None, None))' is an invalid key

In [ ]:
mean_pbv = float(df2["P/BV [Latest] (x)"].iloc[[1]])
sd_pbv = float(df2["P/BV [Latest] (x)"].iloc[[4]])
z_score_pbv = -(df2["P/BV [Latest] (x)"] - mean_pbv) / sd_pbv

0    NaN
1    0.0
2    NaN
3    NaN
4    NaN
5    NaN
Name: P/BV [Latest] (x), dtype: float64